# 1 Exercise 2

Description

In [16]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

from typing import Tuple

In [22]:
def optimal_percentage(price: float, cost: float, holding_cost: float) -> float:
    return (price - cost)/(price + holding_cost)

In [9]:
def root_mean_squared_error(quantity_estimation_vector: np.ndarray, known_optimal_quantity: float) -> float:
    estimation_error: np.ndarray = quantity_estimation_vector - known_optimal_quantity
    squared_error: float = np.square(estimation_error).sum()

    vector_size: int = quantity_estimation_vector.size
    mean_squared_error: float = squared_error/vector_size
    return np.sqrt(mean_squared_error)


In [14]:
def profit_loss_ratio(profit_estimation_vecor: np.ndarray, known_maximum_profit: float) -> float:
    ratio: np.ndarray = (known_maximum_profit - profit_estimation_vecor)/known_maximum_profit
    vector_size = profit_estimation_vecor.size
    return np.absolute(ratio).sum()/vector_size


In [21]:
def normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    mean: float = data_vector.mean()
    var: float = data_vector.var()
    print(mean)
    return (mean, var)

def parametric_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, var = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float =  sts.norm.ppf(optimal_percent, loc=mean, scale=var)

    return inv_cdf_quantity

In [23]:
def log_normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]:
    # Dit moet nog even gecheckt worden, ik heb geen idee of dit is hoe je het berekent. Micha
    mean: float = np.log(data_vector).mean()
    var: float = np.log(data_vector).var()
    return (mean, var)

def parametric_log_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    mean, var = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float = sts.lognorm.ppf(optimal_percent, loc=np.exp(mean), scale=np.exp(var))

    return inv_cdf_quantity

In [24]:
def non_parametric_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    sorted_data_vector: np.ndarray = data_vector.sort()
    vector_size = data_vector.size
    optimal_percent = optimal_percentage(price, cost, holding_cost)
    optimal_arg_value = np.ceil(optimal_percentage * vector_size)

    return data_vector[optimal_arg_value]

In [36]:
def normal_monte_carlo_simulation(number, target_surface, location, scale, price):
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    print(number, target_surface, parametric_normal_optimal_quantity(1 - target_surface, 0, price, data_vector))


In [37]:
def normal():
    location = 100
    scale = 10
    number = [10, 50, 100, 200]
    target_surface = [0.01, 0.05, 0.1, 0.3, 0.7, 0.9, 0.95, 0.99]
    price = 1

    for n in number:
        for t in target_surface:
            normal_monte_carlo_simulation(n, t, location, scale, price)

In [38]:
normal()

10 0.01 -69.6721843324861
10 0.05 41.23082234440761
10 0.1 33.80525746649745
10 0.3 48.1564768002905
10 0.7 150.67369183116435
10 0.9 184.75214251823337
10 0.95 319.43681112683157
10 0.99 398.8870608743571
50 0.01 -81.42751585108905
50 0.05 -40.339439499044644
50 0.1 -41.176473902589194
50 0.3 46.742356173071556
50 0.7 153.8451356046978
50 0.9 267.84292750004334
50 0.95 280.9763352407508
50 0.99 282.8356291065815
100 0.01 -94.73378257337085
100 0.05 -40.71879000500742
100 0.1 -32.57928918946011
100 0.3 39.87159552670605
100 0.7 153.49138107134956
100 0.9 174.66037490975197
100 0.95 236.25840603689977
100 0.99 328.0885553445834
200 0.01 -111.9313094373639
200 0.05 -44.418087340416776
200 0.1 -37.95140235356027
200 0.3 42.548895941987105
200 0.7 153.58386841755694
200 0.9 209.92858203172932
200 0.95 264.2200265472246
200 0.99 363.1225365851551
